# Продвинутое машинное обучение: 
## Домашнее задание 2


In [1]:
from collections import defaultdict
from tqdm.notebook import tqdm
import json
import pickle
import pandas as pd
import numpy as np
import sklearn
import scipy
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy.stats.mstats import spearmanr
from scipy.stats.mstats import kendalltau

from sklearn.linear_model import LogisticRegression

## 1. Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). 


In [2]:
tournaments = pickle.load(open('tournaments.pkl', 'rb'))
players = pickle.load(open('players.pkl', 'rb'))
results = pickle.load(open('results.pkl', 'rb'))

Изначально посмотрел, какие данные есть и что они из себя представляют. Попробовал посмотреть случайный турнир

In [3]:
tournaments[5443]

{'id': 5443,
 'name': 'Vilnius Open',
 'dateStart': '2019-05-04T14:00:00+03:00',
 'dateEnd': '2019-05-05T15:00:00+03:00',
 'type': {'id': 2, 'name': 'Обычный'},
 'season': '/seasons/52',
 'orgcommittee': [{'id': 26382,
   'name': 'Рихард',
   'patronymic': 'Юрьевич',
   'surname': 'Пфейфле'},
  {'id': 73380,
   'name': 'Олег',
   'patronymic': 'Олегович',
   'surname': 'Сероухов'}],
 'synchData': None,
 'questionQty': {'1': 15, '2': 15, '3': 15, '4': 15}}

Думал, что пригодится информация о типах турнира, но в итоге я это никак не использовал, но посмотрел.

In [4]:
types_id = dict()
for i in tournaments:
    types_id[tournaments[i]['type']['id']] = tournaments[i]['type']['name']
    
types_id

{2: 'Обычный',
 3: 'Синхрон',
 4: 'Региональный',
 10: 'Марафон',
 8: 'Асинхрон',
 1: 'Неизвестный',
 6: 'Строго синхронный',
 5: 'Общий зачёт'}

Решил попарсить данные и собрать датафрейм, чтобы удобнее было анализировать. Как то изначально сразу было решено, что буду объединять id игроков по клубам. В итоге в дальнейшем именно этот подход пригодился, а имя и фамилию в принципе можно всегда забрать по id из players.

In [46]:
def build_db_without_players_for_test(tournaments, results, players, year):
    t_names = list()
    t_id = list()
    t_type_id = list()
    t_type_name = list()

    club_id = list()
    club_place = list()
    club_points = list()
    club_names = list()
    club_players = list()

    mask = list()

    none_count = 0
    tournaments_count = 0
    
    drop_flag = False

    for i in tournaments:
        if drop_flag:
            none_count += 1
            drop_flag = False
        if tournaments[i]['dateStart'][:4] == str(year):
            tournaments_count += 1
            mask_flag = True
            for j in results[tournaments[i]['id']]:
                try:
                    if sum(tournaments[i]['questionQty'].values()) != len(j['mask']) and len(results[tournaments[i]['id']]) != 0:
                        mask_flag = False
                        break
                except Exception:
                    mask_flag = False
                    none_count += 1
                    break
            if len(results[tournaments[i]['id']]) == 0:
                mask_flag = False                    
            if mask_flag is True:
                for j in results[tournaments[i]['id']]:
                    try:
                        tmp_mask = int(j['mask'])
                        mask.append(j['mask'])
                        club_points.append(j['questionsTotal'])
                        club_place.append(j['position'])
                    except Exception:
                        drop_flag = True
                        continue

                    t_type_id.append(tournaments[i]['type']['id'])
                    t_type_name.append(tournaments[i]['type']['name'])

                    t_names.append(tournaments[i]['name'])
                    t_id.append(tournaments[i]['id'])
                    club_id.append(j['team']['id'])
                    club_names.append(j['team']['name'])

                    ans = list()
                    for k in j['teamMembers']:
                        ans.append(k['player']['id'])

                    club_players.append(ans)   
                

    db_f = pd.DataFrame(np.c_[t_id, t_names, t_type_id, t_type_name, club_id, club_names, club_players, mask, club_points, club_place])
    db_f.columns = ['tournament id', 'tournament name', 'type id', 'type name', 'club id', 'club name', 'players id', 'mask', 'points', 'place']

    print('Выкинули турниров: ', none_count)
    return db_f, t_id

In [6]:
db_2019, tournaments_ids = build_db_without_players_for_test(tournaments, results, players, 2019)
db_2019.head()

C:\Users\MishaYuliaAlina\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
C:\Users\MishaYuliaAlina\Anaconda3\lib\site-packages\numpy\lib\index_tricks.py:388: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  newobj = array(item, copy=False, subok=True, ndmin=ndmin)


Выкинули турниров:  77


,tournament id,tournament name,type id,type name,club id,club name,players id,mask,points,place
0,4772,Синхрон северных стран. Зимний выпуск,3,Синхрон,45556,Рабочее название,"[6212, 18332, 18036, 22799, 15456, 26089]",111111111011111110111111111100010010,28,1
1,4772,Синхрон северных стран. Зимний выпуск,3,Синхрон,1030,Сборная Бутана,"[1585, 40840, 1584, 10998, 16206]",111111111011110100101111011001011010,25,5.5
2,4772,Синхрон северных стран. Зимний выпуск,3,Синхрон,4252,Ять,"[23513, 18168, 21060, 35850, 31332, 10187]",111111111011110101101111001011110000,25,5.5
3,4772,Синхрон северных стран. Зимний выпуск,3,Синхрон,5444,Эйфью,"[36742, 28939, 54289, 15381, 27375]",101111101111111110001101011001111010,25,5.5
4,4772,Синхрон северных стран. Зимний выпуск,3,Синхрон,40931,Здоровенный Я,"[28689, 17720, 30597, 12400, 26988, 69476]",111111101011111101000111001001111110,25,5.5


В первом приближении выкинул 77 турниров, маски которых было невозможно привести к целочисленному типу - то есть они либо были пустые, либо содержали символы x и ?

Понятно, что скорее всего кроме некорректных масок в данных могли быть еще какие то проблемы, и я решил уже попарсить все, привести все данные к парам игрок-вопрос и поставить им в соответствие целевое значение. В итоге получил магическое число QUESTIONS_SIZE = 13454728 - такое количество пар игрок-вопрос за 2019 год предполагалось обучать в дальнейшем

## 2. Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков.

Данных получилось очень много. Изначально хотел обучать на основе датафрейма pandas, но в итоге мой ноутбук с 8 Гб ОЗУ просто не справлялся, на ПК с 16 Гб тоже возникли проблемы. Если для 2020 года еще более или менее что то получалось, то в 2019 году был слишком большой объем данных. В итоге решил использоваль разреженные матрицы. 

Идея заключается в следующем (использовались подходы из лекции про сусликов и болезни:)) - сделать one_hot_encoder для игроков и для вопросов и обучаться на их основе. Но так как к сожалению некоторые вопросы пришлось выкинуть, да и id игроков шли не по порядку, решил сделать два словаря вопросов и игроков, чтобы ставить с помощью них в соответствие в каком столбце матрицы игрока и вопрос нужно распологать. 

Кроме этого еще посчитал массив tour_vs_question, который хранит в себе информацию, сколько в каком турнире было вопросов и количество в сумме вопросов до этого турнира, чтобы было удобнее располагать в матрице вопросы. 

Работал с json в следующем формате: неудачная попытка спарсить означала, что необходимые данные отсутствуют и турнир полностью пропускается. 

In [7]:
QUESTIONS_SIZE = 13454728

def build_db_with_players_and_targets(tournaments, results, players, year):


    questions_vs_torneo = dict()
    
    t_names = list()
    t_id = list()
    t_type_id = list()
    t_type_name = list()

    club_id = list()
    club_place = list()
    club_points = list()
    club_names = list()

    player_id = list()
    player_name = list()
    player_patronymic = list()
    player_surname = list()
    question_num = list()
    
    target = list()

    mask = list()
    
    pos_id = list()

    none_count = 0
    question_num_count = 0
    _id_count = 0
    
    questions_count = dict()
    
    questions_ = np.load('questions_2019_2020_ids_09_04.npy')
    players_ = np.load('players_2019_2020_ids_09_04.npy')
    
    with open('tour_vs_question_09_04.sav', 'rb') as f:
        tour_vs_question = pickle.load(f)    
        
    questions_dict = {
        questions_[i] : i for i in range(len(questions_))
    }
    players_dict = {
        players_[i] : i for i in range(len(players_))    
    }    
    
    ohe_hot_player = scipy.sparse.lil_matrix((QUESTIONS_SIZE, np.max(players_)))
    ohe_hot_questions = scipy.sparse.lil_matrix((QUESTIONS_SIZE, np.max(questions_)))

    for i in tqdm(tournaments):
        if tournaments[i]['dateStart'][:4] == str(year):
            mask_flag = True
            for j in results[tournaments[i]['id']]:
                try:
                    tmp_mask = int(j['mask'])
                    if sum(tournaments[i]['questionQty'].values()) != len(j['mask']) and len(results[tournaments[i]['id']]) != 0:
                        mask_flag = False
                        break
                except Exception:
                    mask_flag = False
                    break
            if len(results[tournaments[i]['id']]) == 0:
                mask_flag = False
            if mask_flag is True:
                question_num_position = tour_vs_question[tournaments[i]['id']][1] - tour_vs_question[tournaments[i]['id']][0]
                for j in results[tournaments[i]['id']]:
                    for k in j['teamMembers']:
                        try:                      
                            for l in range(len(j['mask'])):
                                
                                #try!!!
                                tmp_target = int(j['mask'][l])
                                tmp_mask = int(j['mask'])
                                tmp_club_points = j['questionsTotal']
                                tmp_club_place = j['position']
                                tmp_t_names = tournaments[i]['name']
                                tmp_t_id = tournaments[i]['id']
                                tmp_club_id = j['team']['id']
                                tmp_club_names = j['team']['name']
                                tmp_player_id = players[k['player']['id']]['id']
                                
                                
                                #если все распарсилось, то заполняем!
                                target.append(tmp_target)
                                pos_id.append(_id_count)


                                
                                ohe_hot_player[_id_count, players_dict[tmp_player_id]] = 1
                                ohe_hot_questions[_id_count, questions_[l + question_num_position]] = 1
                                _id_count += 1                                

                                
                                club_id.append(tmp_club_id)


                                question_num.append(l + question_num_position)
                                t_id.append(tmp_t_id)
                                
                                questions_vs_torneo[l + question_num_position] = tmp_t_id
                                player_id.append(tmp_player_id)
                                try:
                                    questions_count[tmp_player_id] += 1
                                except KeyError:
                                    questions_count[tmp_player_id] = 0
                                    

                        except Exception:
                            none_count += 1
                            continue
                try:
                    question_num_count += len(j['mask']) 
                    tour_vs_question[tournaments[i]['id']] = (len(j['mask']), question_num_count)
                except Exception:
                    continue
    
    return questions_count, questions_vs_torneo, ohe_hot_player, ohe_hot_questions, tour_vs_question, pos_id, question_num, t_id, t_type_id, t_type_name, club_id, club_names, player_id, player_name, player_patronymic, player_surname, club_points, club_place, target

In [8]:
questions_count, questions_vs_torneo, ohe_hot_player, ohe_hot_questions, tour_vs_question, pos_id, question_num, t_id, t_type_id, t_type_name, club_id, club_names, player_id, player_name, player_patronymic, player_surname, club_points, club_place, target = build_db_with_players_and_targets(tournaments, results, players, 2019)

Загружаем полученные заранее словари, они пригодятся потом, в том числе при обращении к данным игроков. Полученные матрицы объединил в одну для удобства.

In [9]:
questions_ = np.load('questions_2019_2020_ids_09_04.npy')
players_ = np.load('players_2019_2020_ids_09_04.npy')

ohe_hot_player_csr = scipy.sparse.csr_matrix(ohe_hot_player)
scipy.sparse.save_npz('ohe_hot_player_csr.npz', ohe_hot_player_csr)

ohe_hot_questions = scipy.sparse.csr_matrix(ohe_hot_questions)
scipy.sparse.save_npz('ohe_hot_questions.npz', ohe_hot_questions)

features = hstack((ohe_hot_player_csr, ohe_hot_questions))

Эти словари мне нужны для того, чтобы забирать данные обратно из матрицы.

In [60]:
players_dict_inverse = {
    i : players_[i] for i in range(len(players_))    
}

questions_dict_inverse = {
    i : questions_[i] for i in range(len(questions_))    
}

questions_vs_torneo = {
   question_num[i] :  t_id[i] for i in range(len(question_num))
}

Обучаем логистическую регрессию. В качестве solver сначала выбирал liblinear, но у него есть определенные минусы, из-за которых решить следующие задачи с помощью sklearn было невозможно. 

Для того, чтобы дообучать модель я использовал атрибут warm_start=True, который позволяет это делать. То есть в дальнейшем, при вызове метода fit, веса модели будут инициализироваться значениями полученными предыдущим вызовом функции fit. И вот в liblinear такая возможность отсутствует, за то она есть в saga, так что использовал его. 

Использовал l2 регуляризацию, хотя вроде и без нее получался примерно такой же результат. Вообщем, обучаем модель

In [11]:
baseline_model = LogisticRegression(solver='saga', penalty='l2', warm_start=True).fit(features, np.array(target))

Идея в следующем: веса модели при игроках будут выше, чем чаще игрок отвечал на вопрос. Для вопросов наблюдается обратная тенденция и это логично, чем чаще вопрос оставался без ответа, тем меньший вес у него был. Посмотрим на топ 20 игроков, отсортированных по весам модели

In [12]:
rating = np.argsort(baseline_model.coef_[0][:np.max(players_)])
j = 1
print('#, id, Фамилия, вес модели, количество вопросов в 2019 году')
for i in range(-1, -20, -1):
    try:
        print(j, players_dict_inverse[rating[i]], players[players_dict_inverse[rating[i]]]['surname'], np.round(baseline_model.coef_[0][rating[i]], 4), questions_count[players_dict_inverse[rating[i]]])
        j += 1
    except Exception:
        print(rating[i], '!!!')
        continue

#, id, Фамилия, вес модели, количество вопросов в 2019 году
1 27403 Руссо 4.0806 1756
2 4270 Брутер 3.9394 2200
3 28751 Семушин 3.8836 2986
4 27822 Савченков 3.8549 2545
5 30270 Спешков 3.7593 2896
6 30152 Сорожкин 3.7536 3924
7 18036 Левандовский 3.6204 1067
8 20691 Мереминский 3.6016 1333
9 22799 Николенко 3.5383 1760
10 30260 Спектор 3.5349 196
11 26089 Прокофьева 3.5296 606
12 22935 Новиков 3.5188 1265
13 18332 Либер 3.5075 2819
14 21698 Мосягин 3.5035 792
15 87637 Саксонов 3.4985 854
16 74001 Мокин 3.4911 899
17 13689 Карякин 3.4634 1060
18 37047 Юнгер 3.4581 451
19 56647 Горелова 3.4548 1696


## 6. Бонус: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли. 

Я сразу посчитал количество вопросов, которые каждый игрок сыграл в 2019 году в функции build_db_with_players_and_targets и вернул их в словаре questions_count и и использовал его при выводе. 

Обращает на себя внимание человек по фамилии Спектор - сыграл всего 196 вопросов, но модель его оценила очень высоко. В реальном рейтинга на 2 янвваря 2020 года он занимал 1561 позицию, хотя в 2020 году пошел заметно выше.

В любом случае это явный outlier, среди игроков топ 20. Позже можно посмотреть то же самое и для модели, полученной на основе ЕМ-алгоритма.

## 5. А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)?

Cразу, забегая вперед, решил построить рейтинг вопросов и турниров по их сложности, раз уж веса были получены на baseline модели. Для этого сортирую вопросы по весам и, используя словари questions_dict_inverse и questions_vs_torneo, забираю их из матрицы весов. Дальше ставлю их в соответствие турнирам, в которых они были разыграны 

In [13]:
tournament_range = defaultdict(list)

drop_count = 0

questions_rating = np.argsort(baseline_model.coef_[0][np.max(players_):])


for i in range(len(questions_rating)):
    try:
        tournament_range[questions_vs_torneo[questions_dict_inverse[questions_rating[i]]]].append(baseline_model.coef_[0][np.max(players_) + questions_dict_inverse[questions_rating[i]]])
    except Exception:
        drop_count += 1
        continue

In [14]:
result = list()
for i in tournament_range.keys():
    result.append([np.sum(tournament_range[i]) / len(tournament_range[i]), i])
        
result.sort(key = lambda i: i[0], reverse = False)

Для оценки турниров, суммировал веса вопросов и потом делил на их количество в турнире, чтобы усреднить. Вот так выглядит топ-20 самых сложных турниров:

In [15]:
print('#, Название турнира, id, суммарный вес модели')
for i in range(20):
    print(i + 1, tournaments[result[i][1]]['name'], result[i][1], np.round(result[i][0], 4))

#, Название турнира, id, суммарный вес модели
1 Угрюмый Ёрш 5928 -2.2695
2 Первенство правого полушария 5159 -2.0433
3 Записки охотника 5587 -1.7667
4 Кубок городов 5025 -1.6978
5 Ускользающая сова 5083 -1.6576
6 Знание – Сила VI 5693 -1.6471
7 Чемпионат России 5465 -1.534
8 Чемпионат Минска. Лига А. Тур четвёртый 5515 -1.5312
9 Чемпионат Мира. Этап 2 Группа С 5943 -1.5195
10 All Cats Are Beautiful 5741 -1.5066
11 Чемпионат Мира. Этап 2. Группа В 5942 -1.5004
12 Антибинго 5161 -1.4546
13 Кубок Москвы 5795 -1.4437
14 Львов зимой. Адвокат 5421 -1.421
15 Чемпионат Мира. Этап 1. Группа С 5940 -1.4081
16 Ра-II: синхрон "Борского корабела" 5098 -1.3984
17 Зеркало мемориала памяти Михаила Басса 5930 -1.3909
18 Кубок Кайнозоя 5501 -1.3699
19 Чемпионат Мира. Этап 3. Группа В 5946 -1.3645
20 Чемпионат Мира. Финал. Группа С 5950 -1.3347


А так топ-20 самых легких:

In [16]:
print('#, Название турнира, id, суммарный вес модели')
for i in range(-1, -20, -1):
    print(-i, tournaments[result[i][1]]['name'], result[i][1], np.round(result[i][0], 4))

#, Название турнира, id, суммарный вес модели
1 (а)Синхрон-lite. Лига старта. Эпизод V 5013 2.4241
2 (а)Синхрон-lite. Лига старта. Эпизод III 5009 2.2278
3 (а)Синхрон-lite. Лига старта. Эпизод IX 5702 2.2012
4 (а)Синхрон-lite. Лига старта. Эпизод VII 5698 2.1871
5 Школьный Синхрон-lite. Выпуск 2.5 5012 2.0777
6 Студенческий чемпионат Калининградской области 5457 2.0095
7 (а)Синхрон-lite. Лига старта. Эпизод IV 5011 1.9839
8 (а)Синхрон-lite. Лига старта. Эпизод VI 5313 1.9517
9 Школьная лига. I тур. 5936 1.935
10 (а)Синхрон-lite. Лига старта. Эпизод X 5704 1.9029
11 Школьная лига. III тур. 5955 1.8888
12 Малый кубок Физтеха 6008 1.8247
13 Второй тематический турнир имени Джоуи Триббиани 6003 1.7439
14 Межфакультетский кубок МГУ. Отбор №4 5601 1.7374
15 Школьный Синхрон-lite. Выпуск 2.3 5008 1.7347
16 Школьная лига. II тур. 5954 1.7238
17 Школьный Синхрон-lite. Выпуск 3.1 5697 1.7012
18 Синхрон-lite. Выпуск XXX 5729 1.628
19 Лига вузов. IV тур 5855 1.6211


Дальше решил, что можно также использовать веса игроков, которые участвовали в турнире, так как представляется, что чем более высокорейтинговые игроки участвуют в турнирах, тем более они сложные. Решил проверить эту идею следующим образом - из результата вычесть средний вес игроков турнира.

In [17]:
tournaments_2019_ids = list(set(tournaments_ids))
rating_dict = {}
for i in range(len(rating)):
    try:
        rating_dict[players_dict_inverse[rating[i]]] = baseline_model.coef_[0][rating[i]]
    except KeyError:
        continue

In [18]:
for k in range(len(result)):
    current = list(db_2019[db_2019['tournament id'] == result[k][1]]['players id'])
    ans = 0
    players_count = 0
    for i in range(len(current)):
        for j in current[i]:
            try:
                ans += rating_dict[j]
            except KeyError:
                ans += 0
                
            players_count += 1
                
    result[k][0] -= (ans / players_count)
    
result.sort(key = lambda i: i[0], reverse = False)    

Всего в 2019 году для обучения использовался 551 турнир, и можно посмотреть как располагаются в уточненном силой игроков рейтинге этапы чемпионата мира и различные школьные турниры

In [19]:
print('#, Название турнира, id, суммарный вес модели')
for i in range(len(result)):
    if tournaments[result[i][1]]['name'].lower().find('мира') != -1:
        print(i + 1, tournaments[result[i][1]]['name'], result[i][1], np.round(result[i][0], 4))

#, Название турнира, id, суммарный вес модели
1 Чемпионат Мира. Этап 2. Группа А 5941 -4.3601
3 Чемпионат Мира. Финал. Группа А 5948 -4.1526
4 Чемпионат Мира. Этап 3. Группа А 5945 -4.1072
6 Чемпионат Мира. Этап 1. Группа А 5938 -3.9689
7 Чемпионат Мира. Этап 2. Группа В 5942 -3.9572
8 Чемпионат Мира. Этап 3. Группа В 5946 -3.8474
12 Чемпионат Мира. Финал. Группа В 5949 -3.5692
13 Чемпионат Мира. Этап 1. Группа В 5939 -3.5241
17 Чемпионат Мира. Этап 1. Группа С 5940 -3.4102
23 Чемпионат Мира. Этап 2 Группа С 5943 -3.201
30 Чемпионат Мира. Этап 3. Группа С 5947 -2.9905
31 Чемпионат Мира. Финал. Группа С 5950 -2.9893


In [20]:
print('#, Название турнира, id, суммарный вес модели')
for i in range(len(result)):
    if tournaments[result[i][1]]['name'].lower().find('школ') != -1:
        print(i + 1, tournaments[result[i][1]]['name'], result[i][1], np.round(result[i][0], 4))

#, Название турнира, id, суммарный вес модели
392 Чемпионат Кипра среди школьников 5749 -0.5796
460 Школьники - взрослым 5645 0.1514
462 Школьнае люстэрка 5489 0.1743
467 Чемпионат школы №1568 5458 0.2355
470 Чемпионат Европы среди школьников 5653 0.2836
490 Школьный чемпионат России 5644 0.6653
492 Чемпионат Беларуси среди школьников 5488 0.709
513 Открытый кубок Дальнего Востока (школьный зачёт) 5635 1.1094
531 Школьный Синхрон-lite. Выпуск 3.2 5699 1.7043
535 Кубок княгини Ольги среди школьников 5791 2.0311
537 Школьный Синхрон-lite. Выпуск 2.4 5010 2.1105
539 Школьный Синхрон-lite. Выпуск 2.3 5008 2.2306
544 Школьный Синхрон-lite. Выпуск 3.1 5697 2.4726
546 Школьный Синхрон-lite. Выпуск 3.3 5701 2.5242
547 Школьный Синхрон-lite. Выпуск 2.5 5012 2.5552
549 Школьная лига. II тур. 5954 2.8756
550 Школьная лига. III тур. 5955 3.0727
551 Школьная лига. I тур. 5936 3.1361


## 3. Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
## предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
## в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


Забираем информацию о турнирах за 2020 год.

In [47]:
db_2020, tournaments_ids = build_db_without_players_for_test(tournaments, results, players, 2020)
db_2020.head()

C:\Users\MishaYuliaAlina\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
C:\Users\MishaYuliaAlina\Anaconda3\lib\site-packages\numpy\lib\index_tricks.py:388: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  newobj = array(item, copy=False, subok=True, ndmin=ndmin)


Выкинули турниров:  43


,tournament id,tournament name,type id,type name,club id,club name,players id,mask,points,place
0,5414,Синхрон северных стран,3,Синхрон,66120,Чатик,"[18490, 116901, 8532, 42346, 123190, 22482]",111111111111111111111110111110101111,33,1
1,5414,Синхрон северных стран,3,Синхрон,312,Социал-демократы,"[19599, 3577, 15051, 25724, 34508, 61400]",111111110111111111111011101010111111,31,2.5
2,5414,Синхрон северных стран,3,Синхрон,2,Афина,"[6482, 32458, 36120, 25882, 34846, 30475]",110111111111111111110111111010111011,31,2.5
3,5414,Синхрон северных стран,3,Синхрон,72338,Невыносимая тяжесть быть Иа,"[135968, 157622, 164213, 102479, 134273, 87648]",111111111111110111111110111010101011,30,5
4,5414,Синхрон северных стран,3,Синхрон,4032,X-promt,"[5007, 7737, 29487, 39152, 1352, 66403]",111111110111111111110111100010111111,30,5


In [49]:
tournaments_2020_ids = list(set(tournaments_ids))
rating_dict = {}
for i in range(len(rating)):
    try:
        rating_dict[players_dict_inverse[rating[i]]] = baseline_model.coef_[0][rating[i]]
    except KeyError:
        continue

Было решено оценить результаты турниров опять с помощью весов игроков, полученных в безлайн модели. Для этого суммировал веса игроков команды и ранжировал команды по ним. Также считал среднеарифметическое и брал вес лучшего игрока. Результаты получаются примерно одинаковые

Сумма весов:

In [23]:
spearm = 0
kendal = 0


for k in tqdm(range(len(tournaments_2020_ids))):
    current = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['players id'])
    current_teams = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['club id'])
    current_place = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['place'])
    weights = list()
    for i in range(len(current)):
        ans = 0
        for j in current[i]:
            try:
                ans += rating_dict[j]
            except KeyError:
                ans += 0
        weights.append(-ans)
        
    spearm += spearmanr(weights, current_place).correlation
    kendal += kendalltau(weights, current_place).correlation
    
print('Корреляция Спирмена: ', spearm / len(tournaments_2020_ids))
print('Корреляция Кендалла: ', kendal / len(tournaments_2020_ids))


Корреляция Спирмена:  0.7700100653811479
Корреляция Кендалла:  0.6151267893570874


Среднее арифметическое:

In [24]:
spearm = 0
kendal = 0

division_zero_delta = 1 ** -6

for k in tqdm(range(len(tournaments_2020_ids))):
    current = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['players id'])
    current_teams = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['club id'])
    current_place = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['place'])
    weights = list()
    for i in range(len(current)):
        ans = 0
        for j in current[i]:
            try:
                ans += rating_dict[j]
            except KeyError:
                ans += 0
        weights.append(-ans / (len(current[i]) + division_zero_delta))
        
    spearm += spearmanr(weights, current_place).correlation
    kendal += kendalltau(weights, current_place).correlation
    
print('Корреляция Спирмена: ', spearm / len(tournaments_2020_ids))
print('Корреляция Кендалла: ', kendal / len(tournaments_2020_ids))


Корреляция Спирмена:  0.7738426972415313
Корреляция Кендалла:  0.6176076709277674


Вес лучшего игрока команды:

In [25]:
spearm = 0
kendal = 0


for k in tqdm(range(len(tournaments_2020_ids))):
    current = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['players id'])
    current_teams = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['club id'])
    current_place = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['place'])
    weights = list()
    for i in range(len(current)):
        ans = -np.inf
        for j in current[i]:
            try:
                if rating_dict[j] > ans:
                    ans = rating_dict[j]
            except KeyError:
                continue
        weights.append(-ans)
        
    spearm += spearmanr(weights, current_place).correlation
    kendal += kendalltau(weights, current_place).correlation
    
print('Корреляция Спирмена: ', spearm / len(tournaments_2020_ids))
print('Корреляция Кендалла: ', kendal / len(tournaments_2020_ids))


Корреляция Спирмена:  0.756189248275136
Корреляция Кендалла:  0.601705590531267


## 4. Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
## предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
## разработайте EM-схему для обучения этой модели, реализуйте её в коде;
## обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


Понятно, что предложенная модель очень сильно упрощенная, так как по имеющимся данным невозможно точно сказать кто именно из игроков ответил на вопрос, в безлайн модели предполагается, что если команда ответила на вопрос, то на него ответил и каждый игрок команды. 

Такое происходит очень редко, ну во всяком случае, наблюдая ЧГК по телевизору, мы чаще всего видим, что появляется несколько идей от пары игроков и вся команда минуту раскручивает эти версии и выбирает какую то из них. Иногда бывает, что кто-то знает факт, о котором идет речь в вопросе и дает ответ без обсуждения. 

Я думаю, что в спортивной версии происходят подобные процессы, в то время как безлайн модель их не учитывает.
Хотелось бы получить модель, которая учитывала, что на вопрос может ответить только несколько игроков из команды. 
Идея заключается в следующем - попробовать использовать ЕМ-алгоритм для решения этой задачи, а именно:
1. В качестве начального состояния можно выбрать веса обученной безлайн модели и попредсказывать на ее основе турниры 2019 года, где предсказанные значения по факту:
$sigma(i,j, M) = y_{cj} $
команда c ответила на вопрос j. Здесь i это i-й игрок, а M - параметры нашей модели.
2. Дальше предположим, что у нас есть некоторая скрытая переменная $z_{ij}$, которая определает, что это именно игрок i ответил на вопрос j. Тогда у нас будет два варианта. В первом случае команда не ответила на вопрос  $y_{cj} = 0$ и соответственно игрок тоже на него не ответил $z_{ij} = 0$. Во втором случае команда ответила на вопрос $y_{cj} = 1$ и необходимо посчитать, что именно этот игрок ответил на этот вопрос. Это можно сделать следующим образом. Посчитаем условную вероятность того, что именно этот игрок ответил на этот вопрос, при условии, что кто то из команды на него ответил:

![equation](https://latex.codecogs.com/png.image?\dpi{110}&space;\inline&space;z_{ij}&space;=&space;\frac{sigma(i,j,&space;M)}{1&space;-&space;\prod\limits_{c}1&space;-&space;sigma(i_c,j,&space;M))

3. После этого можно посмотреть превышен ли порог, при котором можно сказать ответил игрок на вопрос или нет, по умолчанию 0.5 как и в безлайн модели. В этом будет заключается Е-шаг алгоритма.
4. На основе полученных значений на М-шаге можно снова обучить модель и далее повторять последовательно шаги 2-3.

Выполняем шаг 1:

In [26]:
em_target = baseline_model.predict_proba(features)[:,1]

Выполняем шаги 2 и 3. Сначала считаем условные вероятности, а потом сравниваем их с порогом. Первый порог THRESHOLD отвечает за предсказания на E шаге, второй THRESHOLD_TRUE, за предсказания полученные М-шаге для того, чтобы проверять ответила команда на вопрос или нет. 

In [27]:
current_id = club_id[0]
ans = list()
ans2 = [1] * tour_vs_question[t_id[i]][0]
count = 0
THRESHOLD = 0.5
THRESHOLD_TRUE = 0.5

YES_ANSWER = 1
NO_ANSWER = 0

new_em_target = list()
current_len = tour_vs_question[t_id[i]][0]

for i in tqdm(range(len(em_target))):
    question_num_position = tour_vs_question[t_id[i]][1] - tour_vs_question[t_id[i]][0]

    
    if club_id[i] == current_id:
        ans.append(em_target[i])
        ans2[i % current_len] *= (1 - em_target[i])
        count += 1
    else:
        step = int(count / tour_vs_question[t_id[i]][0])
        
        for j in range(len(ans)):

            if(((ans[j] / (1 - ans2[j % current_len]))  > THRESHOLD) and ans[j] > THRESHOLD_TRUE):
                new_em_target.append(YES_ANSWER)
            else:
                new_em_target.append(NO_ANSWER)

        
        current_id = club_id[i]
        ans = list()
        count = 0
        ans2 = [1] * tour_vs_question[t_id[i]][0]
        current_len = tour_vs_question[t_id[i]][0]
        ans.append(em_target[i])

step = int(count / tour_vs_question[t_id[i]][0])
        
for j in range(len(ans)):

    if(((ans[j] / (1 - ans2[j % current_len])) > THRESHOLD) and ans[j] > THRESHOLD_TRUE):
        new_em_target.append(YES_ANSWER)
    else:
        new_em_target.append(NO_ANSWER)
   

C:\Users\MishaYuliaAlina\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: divide by zero encountered in double_scalars


На основе полученных предсказания сделаем М-шаг. За счет использования атрибута warm_start=True, модель не будет перезатирать полученные ранее веса, а начнет их уточнять.

In [28]:
baseline_model.fit(features, new_em_target)

LogisticRegression(solver='saga', warm_start=True)

Посмотрим как изменился рейтинг - игрок Спектор с малым наигрышем вылетел из топ-20.

In [29]:
rating = np.argsort(baseline_model.coef_[0][:np.max(players_)])
j = 1
print('#, id, Фамилия, вес модели, количество вопросов в 2019 году')
for i in range(-1, -20, -1):
    try:
        print(j, players_dict_inverse[rating[i]], players[players_dict_inverse[rating[i]]]['surname'], np.round(baseline_model.coef_[0][rating[i]], 4), questions_count[players_dict_inverse[rating[i]]])
        j += 1
    except Exception:
        print(rating[i], '!!!')
        continue

#, id, Фамилия, вес модели, количество вопросов в 2019 году
1 27403 Руссо 13.1431 1756
2 4270 Брутер 12.9163 2200
3 28751 Семушин 12.8875 2986
4 27822 Савченков 12.8275 2545
5 30152 Сорожкин 12.5711 3924
6 30270 Спешков 12.4743 2896
7 20691 Мереминский 12.2551 1333
8 18036 Левандовский 12.0227 1067
9 22935 Новиков 11.9979 1265
10 87637 Саксонов 11.991 854
11 74001 Мокин 11.972 899
12 37047 Юнгер 11.9634 451
13 56647 Горелова 11.9319 1696
14 18332 Либер 11.9164 2819
15 7008 Гилёв 11.9088 3420
16 24384 Пашковский 11.8981 1223
17 23737 Островский 11.8912 2087
18 13689 Карякин 11.8568 1060
19 21698 Мосягин 11.85 792


Далее можно повторять эти шаги, для того, чтобы уточнить полученные веса

## 6. Бонус: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли. 

Хотя Спектор и вылетел из топ-20, позиции Юнгера только укрепились, дальше, если повторить Е и М шаги он пойдет еще выше в топ-10.

Можно снова попредсказывать турниры за 2020 год - сильно результат не изменился

In [30]:
tournaments_2020_ids = list(set(tournaments_ids))
rating_dict = {}
for i in range(len(rating)):
    try:
        rating_dict[players_dict_inverse[rating[i]]] = baseline_model.coef_[0][rating[i]]
    except KeyError:
        continue

In [31]:
spearm = 0
kendal = 0


for k in tqdm(range(len(tournaments_2020_ids))):
    current = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['players id'])
    current_teams = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['club id'])
    current_place = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['place'])
    weights = list()
    for i in range(len(current)):
        ans = 0
        for j in current[i]:
            try:
                ans += rating_dict[j]
            except KeyError:
                ans += 0
        weights.append(-ans)
        
    spearm += spearmanr(weights, current_place).correlation
    kendal += kendalltau(weights, current_place).correlation
    
print(spearm / len(tournaments_2020_ids))
print(kendal / len(tournaments_2020_ids))


0.76000415014037
0.6029662907680069


Интересно насколько точен предсказанный топ-100 модели от реального рейтинга. Можно взять топ-100 от 2 января 2020 года и посмотреть количество пересечений предсказанного рейтинга с реальным:

In [32]:
real_top_100 = [28751,27822,30270,27403,4270,18332,23178,33620,116713,6212,15456,7008,28689,19915,22799,95884,32901,23321,90915,6482,3207,32458,2421,10695,110920,86775,18036,74382,23737,42511,23020,33032,26089,56000,27499,15811,29800,36120,505,34365,96330,40877,26640,113703,24341,31315,33456,86795,12313,21233,12747,31493,38458,34846,116757,72211,70750,88067,29989,60149,26173,16837,38018,20814,54035,5328,17296,29531,24384,7890,5990,37761,15595,2954,15217,27701,25819,54136,80593,14786,31190,28260,54696,66764,80897,37718,31898,25177,34936,3438,10586,33792,116758,33509,20691,25882,98594,85216,4121]

real_top_100_dict = {
    real_top_100[i] : i for i in range(len(real_top_100))
}

rating = np.argsort(baseline_model.coef_[0][:np.max(players_)])
j = 1
predicted_top_100 = list()
for i in range(-1, -100, -1):
    predicted_top_100.append(players_dict_inverse[rating[i]])
    
none_count = 0
for i in predicted_top_100:
    try:
        real_top_100_dict[i]
    except KeyError:
        none_count += 1
        
print('Количество пересечений с реальным рейтингом топ-100 на 2 января 2020 года: ', 100 - none_count)
        

Количество пересечений с реальным рейтингом топ-100 на 2 января 2020 года:  50


Получилось 46 пересечений - с учетом базы игроков на 60196 человек кажется, что это не самый плохой результат

Можно посмотреть как выглядит топ турниров , полученных на основе ЕМ алгоритма. 

In [35]:
tournament_range = defaultdict(list)

drop_count = 0

questions_rating = np.argsort(baseline_model.coef_[0][np.max(players_):])


for i in range(len(questions_rating)):
    try:
        tournament_range[questions_vs_torneo[questions_dict_inverse[questions_rating[i]]]].append(baseline_model.coef_[0][np.max(players_) + questions_dict_inverse[questions_rating[i]]])
    except Exception:
        drop_count += 1
        continue

In [36]:
result = list()
for i in tournament_range.keys():
    result.append([np.sum(tournament_range[i]) / len(tournament_range[i]), i])
        
result.sort(key = lambda i: i[0], reverse = False)

In [39]:
print('#, Название турнира, id, суммарный вес модели')
for i in range(20):
    print(i + 1, tournaments[result[i][1]]['name'], result[i][1], np.round(result[i][0], 4))

#, Название турнира, id, суммарный вес модели
1 Угрюмый Ёрш 5928 -6.517
2 Первенство правого полушария 5159 -5.875
3 Кубок городов 5025 -5.6362
4 Записки охотника 5587 -5.1872
5 Ускользающая сова 5083 -5.1291
6 Чемпионат России 5465 -4.9687
7 Серия Premier. Седьмая печать 5412 -4.875
8 Тихий Донец: омут первый 5996 -4.7863
9 Ра-II: синхрон "Борского корабела" 5098 -4.7672
10 Знание – Сила VI 5693 -4.5646
11 Антибинго 5161 -4.5644
12 All Cats Are Beautiful 5741 -4.4975
13 Чемпионат Мира. Этап 2. Группа В 5942 -4.4656
14 Чемпионат Мира. Этап 3. Группа В 5946 -4.4033
15 Мемориал Дмитрия Коноваленко 5303 -4.3928
16 Чемпионат Мира. Этап 2. Группа А 5941 -4.3532
17 Кубок Москвы 5795 -4.34
18 Чемпионат Минска. Лига А. Тур четвёртый 5515 -4.1821
19 Чемпионат Мира. Финал. Группа В 5949 -4.1204
20 Львов зимой. Адвокат 5421 -4.0488


In [40]:
print('#, Название турнира, id, суммарный вес модели')
for i in range(-1, -20, -1):
    print(-i, tournaments[result[i][1]]['name'], result[i][1], np.round(result[i][0], 4))

#, Название турнира, id, суммарный вес модели
1 (а)Синхрон-lite. Лига старта. Эпизод V 5013 6.5673
2 (а)Синхрон-lite. Лига старта. Эпизод III 5009 6.3593
3 (а)Синхрон-lite. Лига старта. Эпизод IX 5702 6.0317
4 (а)Синхрон-lite. Лига старта. Эпизод VII 5698 5.8723
5 (а)Синхрон-lite. Лига старта. Эпизод IV 5011 5.5643
6 (а)Синхрон-lite. Лига старта. Эпизод X 5704 5.4822
7 Синхрон-lite. Выпуск XXX 5729 5.3699
8 (а)Синхрон-lite. Лига старта. Эпизод VI 5313 5.3403
9 Студенческий чемпионат Калининградской области 5457 4.9588
10 Лига Сибири. VI тур. 5130 4.9526
11 KFC 5511 4.9514
12 Школьный Синхрон-lite. Выпуск 2.5 5012 4.9207
13 Лига Сибири. IV тур. 5128 4.6154
14 Лига вузов. IV тур 5855 4.5792
15 Синхрон-lite. Выпуск XXIX 5728 4.3978
16 Синхрон простых вопросов. Зима 5844 4.3511
17 Второй тематический турнир имени Джоуи Триббиани 6003 4.2308
18 Кубок Оливье 5924 4.1588
19 Лига вузов. III тур 5854 4.0955


## Бонус: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли. Скорее всего, вы увидите, что топ занят игроками, которые сыграли совсем мало вопросов, около 100 или даже меньше; если вы поищете их в официальном рейтинге ЧГК, вы увидите, что это какие-то непонятные ноунеймы. В baseline-модели, скорее всего, такой эффект будет гораздо слабее.

Сначала я попробовал просто выбросить игроков, котрые сыграли меньше 1000 вопросов, для того, чтобы в рейтинге остались действительно опытные игроки. Пар игрок-вопрос получилось порядка 5,5 миллионов - сокращение от бейзлайна почти в три раза. Это не очень честно, так как рейтинг превращается в закрытый клуб. Если игрок не успел наиграть определенное количество вопросов, то он просто не будет никак учитываться. Кроме этого пострадают веса вопросов, а школьные турниры совсем исчезнут из рейтинга.

Тем не менее, можно посмотреть, что получилось

In [7]:
QUESTIONS_SIZE = 5500090

def build_db_with_players_and_targets(tournaments, results, players, year):


    questions_vs_torneo = dict()
    
    t_names = list()
    t_id = list()
    t_type_id = list()
    t_type_name = list()

    club_id = list()
    club_place = list()
    club_points = list()
    club_names = list()

    player_id = list()
    player_name = list()
    player_patronymic = list()
    player_surname = list()
    question_num = list()
    
    target = list()

    mask = list()
    
    pos_id = list()

    none_count = 0
    question_num_count = 0
    _id_count = 0
    
    with open('questions_count.sav', 'rb') as f:
        questions_count = pickle.load(f)    
    
    questions_ = np.load('questions_2019_2020_ids_09_04.npy')
    players_ = np.load('players_2019_2020_ids_09_04.npy')
    
    with open('tour_vs_question_09_04.sav', 'rb') as f:
        tour_vs_question = pickle.load(f)    
        
    questions_dict = {
        questions_[i] : i for i in range(len(questions_))
    }
    players_dict = {
        players_[i] : i for i in range(len(players_))    
    }    
    
    ohe_hot_player = scipy.sparse.lil_matrix((QUESTIONS_SIZE, np.max(players_)))
    ohe_hot_questions = scipy.sparse.lil_matrix((QUESTIONS_SIZE, np.max(questions_)))

    for i in tqdm(tournaments):
        if tournaments[i]['dateStart'][:4] == str(year):
            mask_flag = True
            for j in results[tournaments[i]['id']]:
                try:
                    tmp_mask = int(j['mask'])
                    if sum(tournaments[i]['questionQty'].values()) != len(j['mask']) and len(results[tournaments[i]['id']]) != 0:
                        mask_flag = False
                        break
                except Exception:
                    mask_flag = False
                    break
            if len(results[tournaments[i]['id']]) == 0:
                mask_flag = False
            if mask_flag is True:
                question_num_position = tour_vs_question[tournaments[i]['id']][1] - tour_vs_question[tournaments[i]['id']][0]
                for j in results[tournaments[i]['id']]:
                    for k in j['teamMembers']:
                        try:                      
                            for l in range(len(j['mask'])):
                                
                                #try!!!
                                tmp_target = int(j['mask'][l])
                                tmp_mask = int(j['mask'])
                                tmp_club_points = j['questionsTotal']
                                tmp_club_place = j['position']
                                tmp_t_names = tournaments[i]['name']
                                tmp_t_id = tournaments[i]['id']
                                tmp_club_id = j['team']['id']
                                tmp_club_names = j['team']['name']
                                tmp_player_id = players[k['player']['id']]['id']
                                
                                
                                #если все распарсилось, то заполняем!
                                if (questions_count[tmp_player_id] > 1000):
                                    target.append(tmp_target)
                                    pos_id.append(_id_count)



                                    ohe_hot_player[_id_count, players_dict[tmp_player_id]] = 1
                                    ohe_hot_questions[_id_count, questions_[l + question_num_position]] = 1
                                    _id_count += 1                                


                                    club_id.append(tmp_club_id)


                                    question_num.append(l + question_num_position)
                                    t_id.append(tmp_t_id)

                                    questions_vs_torneo[l + question_num_position] = tmp_t_id
                                    player_id.append(tmp_player_id)

                                    

                        except Exception:
                            none_count += 1
                            continue
                try:
                    question_num_count += len(j['mask']) 
                    tour_vs_question[tournaments[i]['id']] = (len(j['mask']), question_num_count)
                except Exception:
                    continue
    
    return questions_vs_torneo, ohe_hot_player, ohe_hot_questions, tour_vs_question, pos_id, question_num, t_id, t_type_id, t_type_name, club_id, club_names, player_id, player_name, player_patronymic, player_surname, club_points, club_place, target

In [8]:
questions_vs_torneo, ohe_hot_player, ohe_hot_questions, tour_vs_question, pos_id, question_num, t_id, t_type_id, t_type_name, club_id, club_names, player_id, player_name, player_patronymic, player_surname, club_points, club_place, target = build_db_with_players_and_targets(tournaments, results, players, 2019)

In [11]:
ohe_hot_player_csr = scipy.sparse.csr_matrix(ohe_hot_player)
scipy.sparse.save_npz('ohe_hot_player_csr.npz', ohe_hot_player_csr)

ohe_hot_questions = scipy.sparse.csr_matrix(ohe_hot_questions)
scipy.sparse.save_npz('ohe_hot_questions.npz', ohe_hot_questions)

features = hstack((ohe_hot_player_csr, ohe_hot_questions))

In [13]:
baseline_model = LogisticRegression(solver='saga', penalty='l2', warm_start=True).fit(features, np.array(target))

In [28]:
with open('questions_count.sav', 'rb') as f:
    questions_count = pickle.load(f) 
    
rating = np.argsort(baseline_model.coef_[0][:np.max(players_)])
j = 1
print('#, id, Фамилия, вес модели, количество вопросов в 2019 году')
for i in range(-1, -20, -1):
    try:
        print(j, players_dict_inverse[rating[i]], players[players_dict_inverse[rating[i]]]['surname'], np.round(baseline_model.coef_[0][rating[i]], 4), questions_count[players_dict_inverse[rating[i]]])
        j += 1
    except Exception:
        print(rating[i], '!!!')
        continue

#, id, Фамилия, вес модели, количество вопросов в 2019 году
1 27403 Руссо 2.404 1756
2 4270 Брутер 2.2589 2200
3 28751 Семушин 2.1979 2986
4 27822 Савченков 2.1718 2545
5 30270 Спешков 2.0715 2896
6 30152 Сорожкин 2.0683 3924
7 18036 Левандовский 1.9423 1067
8 20691 Мереминский 1.9213 1333
9 22935 Новиков 1.8397 1265
10 22799 Николенко 1.8286 1760
11 18332 Либер 1.8213 2819
12 13689 Карякин 1.7853 1060
13 56647 Горелова 1.773 1696
14 16837 Кудряшова 1.7571 2074
15 16332 Крапиль 1.7557 1318
16 24384 Пашковский 1.7502 1223
17 23737 Островский 1.7396 2087
18 1585 Архангельская 1.7388 1354
19 34846 Чернин 1.736 1345


Из-за того, что в модели теперь просто отсутствует информация о многих игроках, качество предсказания турниров за 2020 год сильно падает

In [50]:
spearm = 0
kendal = 0


for k in tqdm(range(len(tournaments_2020_ids))):
    current = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['players id'])
    current_teams = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['club id'])
    current_place = list(db_2020[db_2020['tournament id'] == tournaments_2020_ids[k]]['place'])
    weights = list()
    for i in range(len(current)):
        ans = 0
        for j in current[i]:
            try:
                ans += rating_dict[j]
            except KeyError:
                ans += 0
        weights.append(-ans)
        
    spearm += spearmanr(weights, current_place).correlation
    kendal += kendalltau(weights, current_place).correlation
    
print(spearm / len(tournaments_2020_ids))
print(kendal / len(tournaments_2020_ids))


0.43797107641269595
0.3179085086781409


За то выросло количество пересечений топ-100 с реальным рейтингом ЧГК - их стало 61.

In [29]:
real_top_100 = [28751,27822,30270,27403,4270,18332,23178,33620,116713,6212,15456,7008,28689,19915,22799,95884,32901,23321,90915,6482,3207,32458,2421,10695,110920,86775,18036,74382,23737,42511,23020,33032,26089,56000,27499,15811,29800,36120,505,34365,96330,40877,26640,113703,24341,31315,33456,86795,12313,21233,12747,31493,38458,34846,116757,72211,70750,88067,29989,60149,26173,16837,38018,20814,54035,5328,17296,29531,24384,7890,5990,37761,15595,2954,15217,27701,25819,54136,80593,14786,31190,28260,54696,66764,80897,37718,31898,25177,34936,3438,10586,33792,116758,33509,20691,25882,98594,85216,4121]

real_top_100_dict = {
    real_top_100[i] : i for i in range(len(real_top_100))
}

rating = np.argsort(baseline_model.coef_[0][:np.max(players_)])
j = 1
predicted_top_100 = list()
for i in range(-1, -100, -1):
    predicted_top_100.append(players_dict_inverse[rating[i]])
    
none_count = 0
for i in predicted_top_100:
    try:
        real_top_100_dict[i]
    except KeyError:
        none_count += 1
        
print('Количество пересечений с реальным рейтингом топ-100 на 2 января 2020 года: ', 100 - none_count)
        

Количество пересечений с реальным рейтингом топ-100 на 2 января 2020 года:  61


Второй вариант более динамический. Идея заключается в следующем - ответы игроков с небольшим наигрышем по вопросам будут зануляться, но информация о них будет аккумулироваться. После того, как количество сыгранных вопросов игроком будет превышать порог, его ответы начнут добавляться в модель, и зануленные данные будут заменены на реальные. Таким образом не будет ситуации, при которой сильный игрок исчезнет из рейтинга. В то же время, у игроков будет стимул (если рейтинг ЧГК является каким то стимулом :)) играть больше турниров. Я опять выбрал порог в 1000 вопросов, но он совсем жесткий, его можно снизить в 2 раза минимум.

In [38]:
QUESTIONS_SIZE = 13454728

def build_db_with_players_and_targets(tournaments, results, players, year):


    questions_vs_torneo = dict()
    
    t_names = list()
    t_id = list()
    t_type_id = list()
    t_type_name = list()

    club_id = list()
    club_place = list()
    club_points = list()
    club_names = list()

    player_id = list()
    player_name = list()
    player_patronymic = list()
    player_surname = list()
    question_num = list()
    
    target = list()

    mask = list()
    
    pos_id = list()

    none_count = 0
    question_num_count = 0
    _id_count = 0
    
    with open('questions_count.sav', 'rb') as f:
        questions_count = pickle.load(f)    
    
    questions_ = np.load('questions_2019_2020_ids_09_04.npy')
    players_ = np.load('players_2019_2020_ids_09_04.npy')
    
    with open('tour_vs_question_09_04.sav', 'rb') as f:
        tour_vs_question = pickle.load(f)    
        
    questions_dict = {
        questions_[i] : i for i in range(len(questions_))
    }
    players_dict = {
        players_[i] : i for i in range(len(players_))    
    }    
    
    ohe_hot_player = scipy.sparse.lil_matrix((QUESTIONS_SIZE, np.max(players_)))
    ohe_hot_questions = scipy.sparse.lil_matrix((QUESTIONS_SIZE, np.max(questions_)))

    for i in tqdm(tournaments):
        if tournaments[i]['dateStart'][:4] == str(year):
            mask_flag = True
            for j in results[tournaments[i]['id']]:
                try:
                    tmp_mask = int(j['mask'])
                    if sum(tournaments[i]['questionQty'].values()) != len(j['mask']) and len(results[tournaments[i]['id']]) != 0:
                        mask_flag = False
                        break
                except Exception:
                    mask_flag = False
                    break
            if len(results[tournaments[i]['id']]) == 0:
                mask_flag = False
            if mask_flag is True:
                question_num_position = tour_vs_question[tournaments[i]['id']][1] - tour_vs_question[tournaments[i]['id']][0]
                for j in results[tournaments[i]['id']]:
                    for k in j['teamMembers']:
                        try:                      
                            for l in range(len(j['mask'])):
                                
                                #try!!!
                                tmp_target = int(j['mask'][l])
                                tmp_mask = int(j['mask'])
                                tmp_club_points = j['questionsTotal']
                                tmp_club_place = j['position']
                                tmp_t_names = tournaments[i]['name']
                                tmp_t_id = tournaments[i]['id']
                                tmp_club_id = j['team']['id']
                                tmp_club_names = j['team']['name']
                                tmp_player_id = players[k['player']['id']]['id']
                                
                                
                                #если все распарсилось, то заполняем!
                                try:
                                    if questions_count[tmp_player_id] > 1000:
                                        target.append(tmp_target)
                                    else:
                                        target.append(0)
                                except KeyError:
                                    target.append(0)
                                    
                                pos_id.append(_id_count)


                                
                                ohe_hot_player[_id_count, players_dict[tmp_player_id]] = 1
                                ohe_hot_questions[_id_count, questions_[l + question_num_position]] = 1
                                _id_count += 1                                

                                
                                club_id.append(tmp_club_id)


                                question_num.append(l + question_num_position)
                                t_id.append(tmp_t_id)
                                
                                questions_vs_torneo[l + question_num_position] = tmp_t_id
                                player_id.append(tmp_player_id)


                        except Exception:
                            none_count += 1
                            continue
                try:
                    question_num_count += len(j['mask']) 
                    tour_vs_question[tournaments[i]['id']] = (len(j['mask']), question_num_count)
                except Exception:
                    continue
    
    return questions_vs_torneo, ohe_hot_player, ohe_hot_questions, tour_vs_question, pos_id, question_num, t_id, t_type_id, t_type_name, club_id, club_names, player_id, player_name, player_patronymic, player_surname, club_points, club_place, target  

In [39]:
questions_vs_torneo, ohe_hot_player, ohe_hot_questions, tour_vs_question, pos_id, question_num, t_id, t_type_id, t_type_name, club_id, club_names, player_id, player_name, player_patronymic, player_surname, club_points, club_place, target = build_db_with_players_and_targets(tournaments, results, players, 2019)

In [40]:
questions_ = np.load('questions_2019_2020_ids_09_04.npy')
players_ = np.load('players_2019_2020_ids_09_04.npy')

ohe_hot_player_csr = scipy.sparse.csr_matrix(ohe_hot_player)
scipy.sparse.save_npz('ohe_hot_player_csr.npz', ohe_hot_player_csr)

ohe_hot_questions = scipy.sparse.csr_matrix(ohe_hot_questions)
scipy.sparse.save_npz('ohe_hot_questions.npz', ohe_hot_questions)

features = hstack((ohe_hot_player_csr, ohe_hot_questions))

In [41]:
baseline_model = LogisticRegression(solver='saga', penalty='l2', warm_start=True).fit(features, np.array(target))

In [42]:
with open('questions_count.sav', 'rb') as f:
    questions_count = pickle.load(f) 
    
rating = np.argsort(baseline_model.coef_[0][:np.max(players_)])
j = 1
print('#, id, Фамилия, вес модели, количество вопросов в 2019 году')
for i in range(-1, -20, -1):
    try:
        print(j, players_dict_inverse[rating[i]], players[players_dict_inverse[rating[i]]]['surname'], np.round(baseline_model.coef_[0][rating[i]], 4), questions_count[players_dict_inverse[rating[i]]])
        j += 1
    except Exception:
        print(rating[i], '!!!')
        continue

#, id, Фамилия, вес модели, количество вопросов в 2019 году
1 27403 Руссо 8.7957 1756
2 4270 Брутер 8.6586 2200
3 28751 Семушин 8.6048 2986
4 27822 Савченков 8.5758 2545
5 30152 Сорожкин 8.4806 3924
6 30270 Спешков 8.4786 2896
7 18036 Левандовский 8.3211 1067
8 20691 Мереминский 8.31 1333
9 18332 Либер 8.23 2819
10 22799 Николенко 8.2258 1760
11 22935 Новиков 8.2252 1265
12 56647 Горелова 8.1729 1696
13 13689 Карякин 8.1667 1060
14 16837 Кудряшова 8.1597 2074
15 16332 Крапиль 8.1441 1318
16 7008 Гилёв 8.1441 3420
17 23737 Островский 8.142 2087
18 24384 Пашковский 8.1361 1223
19 14786 Коврижных 8.1359 2494


In [43]:
real_top_100 = [28751,27822,30270,27403,4270,18332,23178,33620,116713,6212,15456,7008,28689,19915,22799,95884,32901,23321,90915,6482,3207,32458,2421,10695,110920,86775,18036,74382,23737,42511,23020,33032,26089,56000,27499,15811,29800,36120,505,34365,96330,40877,26640,113703,24341,31315,33456,86795,12313,21233,12747,31493,38458,34846,116757,72211,70750,88067,29989,60149,26173,16837,38018,20814,54035,5328,17296,29531,24384,7890,5990,37761,15595,2954,15217,27701,25819,54136,80593,14786,31190,28260,54696,66764,80897,37718,31898,25177,34936,3438,10586,33792,116758,33509,20691,25882,98594,85216,4121]

real_top_100_dict = {
    real_top_100[i] : i for i in range(len(real_top_100))
}

rating = np.argsort(baseline_model.coef_[0][:np.max(players_)])
j = 1
predicted_top_100 = list()
for i in range(-1, -100, -1):
    predicted_top_100.append(players_dict_inverse[rating[i]])
    
none_count = 0
for i in predicted_top_100:
    try:
        real_top_100_dict[i]
    except KeyError:
        none_count += 1
        
print('Количество пересечений с реальным рейтингом топ-100 на 2 января 2020 года: ', 100 - none_count)
        

Количество пересечений с реальным рейтингом топ-100 на 2 января 2020 года:  62


Пересечение с реальным рейтингом стало еще больше - 62 игрока, вместо 46 из бейзлайна.

## Бонус: игроки со временем учатся играть лучше (а иногда бывает и наоборот). А в нашей модели получается, что первые неудачные турниры новичка будут тянуть его рейтинг вниз всю жизнь — это нехорошо, рейтинг должен быть достаточно гибким и иметь возможность меняться даже у игроков, отыгравших сотни турниров. Давайте попробуем этого добиться:


У меня к сожалению не хватило вычислительных ресурсов, но есть идеи, которые можно использовать.

Первая заключается во введении в модель константы forgetting factor, с помощью которой информация о старых турнирах будет со временем затухать.

Второй вариант не такой мягкий - будем просто отсекать данные по определенному порогу - брать информацию за последний год. Такую идею используют в футболе при подсчете коэффициентов УЕФА для стран.

Данные подходы позволят делать рейтинг более гибким, первые неудачи со временем забудутся из-за коэффициента затухания, либо отсекутся по определенному порогу. 

## Выводы

Очень классная задача и очень классная идея применения логистической регрессии для построения рейтинга. Это домашнее задание было для меня очень полезным, я наконец то полностью разобрался с ЕМ алгоритмом и написал попутно, используя лекционнный материал и учебник Бишопа, код для кластеризации с использованием Е и М шагов. Спасибо!